# Regression part b
### Training and Error calculation

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings('ignore')

# clean and standardize data
data = pd.read_csv('../Data/diamonds.csv')

# Remove rows where 'x', 'y', or 'z' is 0
numerical_columns = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
diamonds_data = data[(data['x'] != 0) & (data['y'] != 0) & (data['z'] != 0)]

# Calculate means and standard deviations for standardization
means = diamonds_data[numerical_columns].mean()
stds = diamonds_data[numerical_columns].std()

# Standardize the numerical columns
diamonds_data_standardized = diamonds_data[numerical_columns].copy()
for column in numerical_columns:
    diamonds_data_standardized[column] = (diamonds_data[column] - means[column]) / stds[column]
print(diamonds_data_standardized.head())

      carat     depth     table     price         x         y         z
0 -1.198193 -0.174201 -1.099715 -0.904123 -1.591558 -1.539205 -1.580069
1 -1.240405 -1.361078  1.585973 -0.904123 -1.645157 -1.661998 -1.750880
2 -1.198193 -3.385749  3.376432 -0.903873 -1.502227 -1.460266 -1.750880
3 -1.071556  0.454145  0.243129 -0.902117 -1.368229 -1.319931 -1.295384
4 -1.029344  1.082491  0.243129 -0.901866 -1.243165 -1.214679 -1.124573


In [10]:
# Extract features and target variable
X = diamonds_data_standardized.drop(columns=['price']).values
y = diamonds_data_standardized['price'].values

# Set up parameters for models
# ==============================
# Define the range for the regularization parameter lambda 
lambda_range = np.logspace(-4, 4, 10)

# Define the range for the number of hidden layers in ANN
hidden_layer_options = [(h,) for h in [1, 2, 3, 4]]

# Two-Level Cross-Validation
# ===========================
# Outer fold (K1) and inner fold (K2) set to 10
K1 = K2 = 10
kf_outer = KFold(n_splits=K1, shuffle=True, random_state=42)

# Initialize lists to store results
results = []

# Outer cross-validation loop
for train_outer_idx, test_outer_idx in kf_outer.split(X):
    # Split data into training and testing sets for the outer fold
    X_train_outer, X_test_outer = X[train_outer_idx], X[test_outer_idx]
    y_train_outer, y_test_outer = y[train_outer_idx], y[test_outer_idx]

    # Inner cross-validation loop for model selection
    kf_inner = KFold(n_splits=K2, shuffle=True, random_state=42)
    best_ann_model = None
    best_ridge_model = None
    best_ann_error = float('inf')
    best_ridge_error = float('inf')
    best_hidden_layer = None
    best_lambda = None

    # ANN model selection
    for hidden_layer in hidden_layer_options:
        inner_errors = []
        for train_inner_idx, val_inner_idx in kf_inner.split(X_train_outer):
            X_train_inner, X_val_inner = X[train_inner_idx], X[val_inner_idx]
            y_train_inner, y_val_inner = y[train_inner_idx], y[val_inner_idx]

            # Train the ANN model
            ann = MLPRegressor(hidden_layer_sizes=hidden_layer, max_iter=1000, random_state=42)
            ann.fit(X_train_inner, y_train_inner)

            # Predict and calculate error
            y_val_pred = ann.predict(X_val_inner)
            error = mean_squared_error(y_val_inner, y_val_pred)
            inner_errors.append(error)
        
        # Calculate the average validation error
        avg_error = np.mean(inner_errors)
        if avg_error < best_ann_error:
            best_ann_error = avg_error
            best_hidden_layer = hidden_layer
            best_ann_model = ann

    # Ridge Regression model selection
    for lmbda in lambda_range:
        inner_errors = []
        for train_inner_idx, val_inner_idx in kf_inner.split(X_train_outer):
            X_train_inner, X_val_inner = X[train_inner_idx], X[val_inner_idx]
            y_train_inner, y_val_inner = y[train_inner_idx], y[val_inner_idx]

            # Train the Ridge Regression model
            ridge = Ridge(alpha=lmbda)
            ridge.fit(X_train_inner, y_train_inner)

            # Predict and calculate error
            y_val_pred = ridge.predict(X_val_inner)
            error = mean_squared_error(y_val_inner, y_val_pred)
            inner_errors.append(error)
        
        # Calculate the average validation error
        avg_error = np.mean(inner_errors)
        if avg_error < best_ridge_error:
            best_ridge_error = avg_error
            best_lambda = lmbda
            best_ridge_model = ridge

    # Baseline model: Predicting the mean of y_train_outer
    y_train_mean = np.mean(y_train_outer)
    y_baseline_pred = np.full_like(y_test_outer, y_train_mean)
    baseline_error = mean_squared_error(y_test_outer, y_baseline_pred)

    # Test the best models on the outer test set
    y_ann_pred = best_ann_model.predict(X_test_outer)
    ann_test_error = mean_squared_error(y_test_outer, y_ann_pred)

    y_ridge_pred = best_ridge_model.predict(X_test_outer)
    ridge_test_error = mean_squared_error(y_test_outer, y_ridge_pred)

    # Store the results for this fold
    results.append({
        'fold': len(results) + 1,
        'best_hidden_layer': best_hidden_layer,
        'ann_test_error': ann_test_error,
        'best_lambda': best_lambda,
        'ridge_test_error': ridge_test_error,
        'baseline_error': baseline_error
    })

# Print results
# ==============
print("Fold | Hidden Layer | ANN Test Error | Lambda | Ridge Test Error | Baseline Error")
for result in results:
    print(f"{result['fold']:>4} | {result['best_hidden_layer'][0]:>12} | {result['ann_test_error']:.4f} | {result['best_lambda']:.4e} | {result['ridge_test_error']:.4f} | {result['baseline_error']:.4f}")

# Conclusion: Compare ANN, Ridge, and Baseline
# =============================================
ann_errors = [result['ann_test_error'] for result in results]
ridge_errors = [result['ridge_test_error'] for result in results]
baseline_errors = [result['baseline_error'] for result in results]

mean_ann_error = np.mean(ann_errors)
mean_ridge_error = np.mean(ridge_errors)
mean_baseline_error = np.mean(baseline_errors)

print("\nAverage Errors:")
print(f"ANN: {mean_ann_error:.4f}")
print(f"Ridge: {mean_ridge_error:.4f}")
print(f"Baseline: {mean_baseline_error:.4f}")


Fold | Hidden Layer | ANN Test Error | Lambda | Ridge Test Error | Baseline Error
   1 |            2 | 0.1217 | 1.6681e+02 | 0.1256 | 0.9676
   2 |            2 | 0.1293 | 1.6681e+02 | 0.1444 | 1.0494
   3 |            2 | 0.1205 | 1.6681e+02 | 0.1319 | 0.9263
   4 |            2 | 0.1250 | 1.6681e+02 | 0.1521 | 1.0162
   5 |            2 | 0.1288 | 1.6681e+02 | 0.1409 | 1.0044
   6 |            2 | 0.1181 | 1.6681e+02 | 0.1380 | 1.0204
   7 |            2 | 0.1355 | 1.6681e+02 | 0.1495 | 0.9810
   8 |            2 | 0.1337 | 1.6681e+02 | 0.1485 | 1.0489
   9 |            2 | 0.1232 | 1.6681e+02 | 0.1345 | 0.9413
  10 |            2 | 0.1543 | 1.6681e+02 | 0.1583 | 1.0451

Average Errors:
ANN: 0.1290
Ridge: 0.1424
Baseline: 1.0001


### T-test

In [13]:
import numpy as np
from scipy import stats

# Errors from each model based on the cross-validation results
ann_errors = np.array([0.1217, 0.1293, 0.1205, 0.1250, 0.1288, 0.1181, 0.1355, 0.1337, 0.1232, 0.1543])
linear_regression_errors = np.array([0.1256, 0.1444, 0.1319, 0.1521, 0.1409, 0.1380, 0.1495, 0.1485, 0.1345, 0.1583])
baseline_errors = np.array([0.9676, 1.0494, 0.9263, 1.0162, 1.0044, 1.0204, 0.9810, 1.0489, 0.9413, 1.0451])

# Function to perform paired t-test and calculate confidence interval
def paired_t_test_and_ci(errors1, errors2, alpha=0.05):
    # Perform paired t-test
    t_stat, p_value = stats.ttest_rel(errors1, errors2)
    
    # Calculate mean difference and standard error of the difference
    differences = errors1 - errors2
    mean_diff = np.mean(differences)
    se_diff = stats.sem(differences)
    
    # Calculate confidence interval
    confidence_interval = (mean_diff - stats.t.ppf(1 - alpha / 2, df=len(differences) - 1) * se_diff,
                          mean_diff + stats.t.ppf(1 - alpha / 2, df=len(differences) - 1) * se_diff)
    
    return t_stat, p_value, confidence_interval

# Pairwise comparisons
# 1. ANN vs Linear Regression
t_stat_ann_lr, p_value_ann_lr, ci_ann_lr = paired_t_test_and_ci(ann_errors, linear_regression_errors)

# 2. ANN vs Baseline
t_stat_ann_baseline, p_value_ann_baseline, ci_ann_baseline = paired_t_test_and_ci(ann_errors, baseline_errors)

# 3. Linear Regression vs Baseline
t_stat_lr_baseline, p_value_lr_baseline, ci_lr_baseline = paired_t_test_and_ci(linear_regression_errors, baseline_errors)

# Print results
print("Pairwise t-test Results:\n")
print(f"ANN vs Linear Regression:\n  t-statistic = {t_stat_ann_lr:.4f}, p-value = {p_value_ann_lr:.4e}, 95% CI = {ci_ann_lr}\n")
print(f"ANN vs Baseline:\n  t-statistic = {t_stat_ann_baseline:.4f}, p-value = {p_value_ann_baseline:.4e}, 95% CI = {ci_ann_baseline}\n")
print(f"Linear Regression vs Baseline:\n  t-statistic = {t_stat_lr_baseline:.4f}, p-value = {p_value_lr_baseline:.4e}, 95% CI = {ci_lr_baseline}\n")

# Conclusion based on p-values and confidence intervals
if p_value_ann_lr < 0.05:
    print("There is a significant difference between ANN and Linear Regression.")
else:
    print("There is no significant difference between ANN and Linear Regression.")

if p_value_ann_baseline < 0.05:
    print("ANN significantly outperforms the Baseline model.")
else:
    print("There is no significant difference between ANN and the Baseline model.")

if p_value_lr_baseline < 0.05:
    print("Linear Regression significantly outperforms the Baseline model.")
else:
    print("There is no significant difference between Linear Regression and the Baseline model.")


Pairwise t-test Results:

ANN vs Linear Regression:
  t-statistic = -6.1564, p-value = 1.6743e-04, 95% CI = (np.float64(-0.018269130071275473), np.float64(-0.008450869928724526))

ANN vs Baseline:
  t-statistic = -68.6364, p-value = 1.4945e-13, 95% CI = (np.float64(-0.8997585728156408), np.float64(-0.8423414271843592))

Linear Regression vs Baseline:
  t-statistic = -71.0297, p-value = 1.0984e-13, 95% CI = (np.float64(-0.8850057587608299), np.float64(-0.8303742412391703))

There is a significant difference between ANN and Linear Regression.
ANN significantly outperforms the Baseline model.
Linear Regression significantly outperforms the Baseline model.
